<a href="https://colab.research.google.com/github/ucfilho/SwarmCantStopNow/blob/main/Conference_2021/ABCOptmi_filter_benchmark_sphere_A1_pop_50_ite_100_RTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from random import randint
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
!git clone https://github.com/ucfilho/Raianars_Routines #clonar do Github
%cd Raianars_Routines

fatal: destination path 'Raianars_Routines' already exists and is not an empty directory.
/content/Raianars_Routines


In [3]:
############################### Função
import FunctionsNew
############################### Função
dim=10 # 
fchoice='Sphere'
function=FunctionsNew.Sphere
MAX = []
MIN = []

for i in  range(dim):
  MAX.append(5.12)
  MIN.append(-5.12)


In [4]:
def CalcFit(fun):
    result=0
    if(fun>=0):
      result=1/(fun+1)
    else:
      result=1+fabs(fun)
    return result

In [5]:
def BestSource(GlobMin,GlobPars,Foods):
  FoodNumber=len(Foods[:,0])
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    if (f[i]<GlobMin):
      GlobMin=f[i]
      for j in range(D):
        GlobPars[j]=Foods[i,j]
  return GlobMin,GlobPars,Foods

In [6]:
def init(index,Foods,trial,f,fitness,MIN,MAX):
  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  solution=np.zeros(D)
  for j in range(D):
    r=np.random.random()
    Foods[index,j]=r*(MAX[j]-MIN[j])+MIN[j]
    solution[j]=Foods[index,j]
    
  f[index]=function(solution)
  fitness[index]=CalcFit(f[index])
  trial[index]=0
  
  return Foods,trial,f,fitness

In [7]:
def initial(fitness,trial,f,Foods,GlobMin,GlobPars,MIN,MAX):
  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  for i in range(FoodNumber):
    Foods,trial,f,fitness=init(i,Foods,trial,f,fitness,MIN,MAX) 
  GlobMin=f[0]
  for i in range(D):
    GlobPars[i]=Foods[0,i]
  return  f,Foods,GlobMin,GlobPars


In [8]:
def EmployedBees(trial,Foods,MIN,MAX):
  FoodNumber=len(Foods[:,0])
  NP=FoodNumber
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    r = np.random.random()
    par2chan= int(r*D)
    r = np.random.random()
    neighbour=int(r*D)
    if(neighbour >= NP):
      neighbour=NP-1
      
    while(neighbour==i):
      r = np.random.random()
      neighbour=int(r*FoodNumber)
      if(neighbour >= NP):
        neighbour=NP-1
    for j in range(D):
      solution[j]=Foods[i,j]

    r = np.random.random()
    A=Foods[i,par2chan]
    B=Foods[neighbour,par2chan]
    solution[par2chan]=Foods[i,par2chan]+(A-B)*(r-0.5)*2;
    if (solution[par2chan]<MIN[par2chan]):
      solution[par2chan]=MIN[par2chan]
    if (solution[par2chan]>MAX[par2chan]):
      solution[par2chan]=MAX[par2chan]
    ObjValSol=function(solution)
    FitnessSol=CalcFit(ObjValSol)
    
    if(FitnessSol>fitness[i]):
      trial[i]=0
      for j in range(D):
        Foods[i,j]=solution[j]
      f[i]=ObjValSol
      fitness[i]=FitnessSol
    else:
      trial[i]=trial[i]+1 
      
  return trial,Foods

In [9]:
def CalcProb(fitness,prob):
  FoodNumber=len(fitness)
  maxfit=fitness[0]
  for i in range(1,FoodNumber):    
    if (fitness[i]>maxfit):
      maxfit=fitness[i]
  for i in range(FoodNumber):
    prob[i]=(0.9*(fitness[i]/maxfit))+0.1
  return fitness,prob


In [10]:
def OnlookerBees(trial,Foods,MIN,MAX):
  D=len(Foods[0,:])
  solution=np.zeros(D)
  FoodNumber=len(Foods[:,0])
  i=0
  t=0
  while(t<FoodNumber):
    r = np.random.random()
    if(r<prob[i]): 
      r = np.random.random()
      t=t+1
      par2chan=int(r*D)
      r = np.random.random()
      neighbour=int(r*FoodNumber)

      while(neighbour==i):
        r = np.random.random()
        neighbour=int(r*FoodNumber)
      for j in range(D):
        solution[j]=Foods[i,j]

      r = np.random.random()
      A=Foods[i,par2chan]
      B=Foods[neighbour,par2chan]
      solution[par2chan]=A+(A-B)*(r-0.5)*2

      if (solution[par2chan]<MIN[par2chan]):
              solution[par2chan]=MIN[par2chan]
      if (solution[par2chan]>MAX[par2chan]):
              solution[par2chan]=MAX[par2chan]
      ObjValSol=function(solution)
      FitnessSol=CalcFit(ObjValSol)
              
      if(FitnessSol>fitness[i]):
        trial[i]=0
        for j in range(D):
          Foods[i,j]=solution[j]
        f[i]=ObjValSol
        fitness[i]=FitnessSol
      else:
        trial[i]=trial[i]+1
      i=i+1
      if (i==FoodNumber):
        i=0
  return trial,Foods


In [11]:
def ScoutBees(fitness,f,Foods,trial,MIN,MAX):
  FoodNumber=len(Foods[:,0])
  max_trial=0
  for i in range(1,FoodNumber):
    if (trial[i]>trial[max_trial]):
      max_trial=i
  if(trial[max_trial]>=limit):
    Foods,trial,f,fitness=init(max_trial,Foods,trial,f,fitness,MIN,MAX)
  return trial,Foods


In [12]:
'''/*Main program of the ABC algorithm*/'''

NPAR=50 #Numero de fontes de comida
ITE=100 #ITERACOES (maxCycle)

PAR=len(MAX) #NUM DE PARAMETROS A SER OTIMIZADOS
limit=100 # quantas vezes obtem resposta identica antes de encerrar 
runtime=3 # quantas vezes vai rodar para tirar a media


In [13]:

f=np.zeros(NPAR) 
Foods=np.zeros((NPAR,PAR)) 
solution=np.zeros(PAR)
fitness=np.zeros(NPAR)
trial=np.zeros(NPAR)
prob=np.zeros(NPAR)
GlobPars=np.zeros(PAR)
GlobMins=np.zeros(runtime)
GlobMin=1e99


In [14]:
# rule half-half
N_2 = int(NPAR/3)

In [15]:
f1=f[:N_2];f2 = f[N_2:];
Foods1=Foods[:N_2,:];Foods2=Foods[N_2:,:];
solution1= np.zeros(PAR);solution2= np.zeros(PAR)
fitness1=fitness[:N_2];fitness2=fitness[N_2:];
trial1=trial[:N_2];trial2=trial[N_2:]
prob1=prob[:N_2];prob2=prob[N_2:];
GlobPars1=np.zeros(PAR);GlobPars2=np.zeros(PAR);
GlobMins1=np.zeros(runtime);GlobMins2=np.zeros(runtime);
GlobMin1=1e99;GlobMin2=1e99

In [16]:
initial_time = time.time()

for run in range(runtime):
  f1,Foods1,GlobMin1,GlobPars1= initial(fitness1,trial1,f1,Foods1,GlobMin1,GlobPars1,MIN,MAX)
  GlobMin1,GlobPars1,Foods1=BestSource(GlobMin1,GlobPars1,Foods1)

  f2,Foods2,GlobMin2,GlobPars2= initial(fitness2,trial2,f2,Foods2,GlobMin2,GlobPars2,MIN,MAX)
  GlobMin2,GlobPars2,Foods2=BestSource(GlobMin2,GlobPars2,Foods2)

  for iter in range(ITE):
    trial1,Foods1=EmployedBees(trial1,Foods1,MIN,MAX);
    trial2,Foods2=EmployedBees(trial2,Foods2,MIN,MAX)

    fitness1,prob1=CalcProb(fitness1,prob1);
    fitness2,prob2=CalcProb(fitness2,prob2)

    trial1,Foods1=OnlookerBees(trial1,Foods1,MIN,MAX);
    trial2,Foods2=OnlookerBees(trial2,Foods2,MIN,MAX)

    GlobPars2_rec = GlobPars2.copy()
    GlobPars1_rec = GlobPars1.copy()

    GlobMin1,GlobPars1,Foods1=BestSource(GlobMin1,GlobPars1,Foods1);
    GlobMin2,GlobPars2,Foods2=BestSource(GlobMin2,GlobPars2,Foods2)

    trial1,Foods1=ScoutBees(fitness1,f1,Foods1,trial1,MIN,MAX)
    trial2,Foods2=ScoutBees(fitness2,f2,Foods2,trial2,MIN,MAX)

    GlobPars2_rec = GlobPars2.copy()
    GlobPars1_rec = GlobPars1.copy()
    
    Foods1[0,:] = GlobPars2.copy()
    Foods1[1,:] = GlobPars1.copy()

    Foods1[2,:] = (GlobPars1+GlobPars1_rec*2.0)/3
    

    Foods2[0,:] = GlobPars2.copy()
    Foods2[1,:] = GlobPars1.copy()
    Foods2[2,:] = (GlobPars2+GlobPars2_rec*2.0)/3


    if(GlobMin1 > GlobMin2):
      GlobMin1 = GlobMin2
    else:
      GlobMin2 = GlobMin1


  '''
  for j in range(PAR):
    print("GlobalParam[",j+1,"]:", GlobPars1[j])

  '''
  current_time = time.time() -initial_time   
  print("run",run+1,"time =",current_time ,"GBest=",GlobMin2," \n");
  GlobMins2[run]=GlobMin2

mean=np.average(GlobMins2)
sd_mean = np.std(GlobMins2)
best_ans = GlobMins2.min()

print("Means of",runtime,"runs:",mean,"\n")
print("Best ans of",runtime,"runs:",best_ans,"\n")
print("Deviation of",runtime,"runs:",sd_mean,"\n")

run 1 time = 0.2037668228149414 GBest= 0.000119390476876713  

run 2 time = 0.4373347759246826 GBest= 0.00036110313802275917  

run 3 time = 0.6584177017211914 GBest= 0.0003096944273272355  

Means of 3 runs: 0.0002633960140755692 

Best ans of 3 runs: 0.000119390476876713 

Deviation of 3 runs: 0.00010396765504005684 



In [17]:
GlobPars1

array([-3.96172546e-03, -7.53777154e-03,  1.22014787e-03,  3.39511906e-05,
       -7.17216560e-03, -4.31026125e-04,  1.02752446e-02, -1.20461254e-02,
       -4.90123660e-03, -1.96346673e-02])